In [53]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

In [34]:
## Input variables are iVariables

iName = "UTC"
iLocation = "Liverpool"
iLocation_type = "Urban" #Urban, Semi-urban or rural
iSize = 20 #square-metres
iStaff = 0 #not sure how many staff or hours
iStart = "01/04/2019"
iType = "Indoor"
iSystem = ['ZipTowers', 21] #System type, quantity of racks
iLights = ["Example", "water", 210, 200, 18] #Type of Light, cooling type, Qty, Wattage, Hours per day
iPlumbing =['Example',1800, 45] #Type of Plumbing kit, wattage, 1 system per X amount of rack units
iClimate = np.array([['','Control System','CO2 Injector','Dehumidifer','Inline Fans'],
                   ['Quantity', 0,1,1,0],
                   ['Watts',100,60,1350,198],
                   ['Hours on per day', 24,16,18,24]])
iSeedling = np.array([['','Pumps','Lights'],
                   ['Quantity', 1,9],
                   ['Watts',33,54],
                   ['Hours on per day', 1,16]])

iHVAC = np.array([['','Heating','Ventilation'],
                   ['Watts' ,0,0],
                   ['Hours on per day', 1,16]])
iAnnual_rent = 23000 #in outreach
iRTQ = 80
iwater_price = 3.20/1000 #United utilities £3.20 per 1000 UK litres
iwaterstandingcharge =  63.77 #United utilities £63.77 standing charge
ienergy_price = 0.125 #UK-Power 12.5p per kWH
ienergystandingcharge = 85 #£85 standing charge
itax = 0 # Council tax
iinternet = 0 # Cost of internet per month

[['' 'Control System' 'CO2 Injector' 'Dehumidifer' 'Inline Fans']
 ['Quantity' '0' '1' '1' '0']
 ['Watts' '100' '60' '1350' '198']
 ['Hours on per day' '24' '16' '18' '24']]
Control System


In [63]:
def labour(size,staff):
    
    if staff == 0:
        staff = 0.0155004*size 
        #0.00144/sq-ft or 0.00007sq-ft for greenhouses
    else:
        staff = staff        
    if size < 930: #square-meters
        wages = 10229.63*staff #$13348.60 per person
    else:
        size > 930 #square-metres
        wages = 19866.07*staff #$25923.14 per person
    monthly_wages = wages/12
    return(monthly_wages)

In [27]:
labour_wages = labour(iSize, iStaff)
print(labour_wages)

264.27226142


In [35]:
def Energy_Lights(System,Lights,energy_price): #Daily cost of energy
    
    if Lights[0] != "unknown" and Lights[2] != 0:
     
        light_wattage = Lights[2]*Lights[3]/1000 #Lighting kW farm usage *qty x wattage
        kWh_daily_lights = light_wattage*Lights[4] #kWh consumption from lights per day
        lights_daily_cost = kWh_daily_lights*ienergy_price
        return(lights_daily_cost)
    
    elif System[0] == 'unknown' and Lights[0] == 'unknown':
        return('unknown')
        
    else:
        System[0] == 'ZipTower' and Lights[0] == "unknown"
        
        if Lights[1] == "air":
            Lights[2] = System[1]*26 #Number of racks x 26
            Lights[3] = 48 #48W for air-cooled
            light_wattage = Lights[2]*Lights[3]/1000 #Lighting kW farm usage qty x wattage
            kWh_daily_lights = light_wattage*Lights[4] #light wattage and hours on per day
            lights_daily_costs = kWh_daily_lights*energy_price #daily kWh x price

        else:
            Lights[1] == 'water'
            Lights[2] = System[1]*10 #Number of racks x 10
            Lights[3] = 200 #200W for water-cooled
            light_wattage = Lights[2]*Lights[3]/1000 #Lighting kW farm usage qty x wattage
            kWh_daily_lights = light_wattage*Lights[4] #light wattage and hours on per day
            lights_daily_costs = kWh_daily_lights*energy_price #daily kWh x price
            
    return(lights_daily_costs)

In [36]:
test = Energy_Lights(iSystem,iLights)
print(test)

94.5


In [75]:
def Energy_Climate(Climate,energy_price):
    
    Control_kWh_daily = Climate[1,1]*Climate[1,2]*Climate[1,3]/1000 # System control: Quantity x Watts x Hours /1000
    Injector_kWh_daily = Climate[2,1]*Climate[2,2]*Climate[2,3]/1000 # CO2 Injector: Quantity x Watts x Hours /1000
    Dehumidifier_kWh_daily = Climate[3,1]*Climate[3,2]*Climate[3,3]/1000 # Dehumidifier: Quantity x Watts x Hours /1000
    Fans_kWh_daily = Climate[4,1]*Climate[4,2]*Climate[4,3]/1000 # Inline Fans: Quantity x Watts x Hours /1000
    
    Climate_energy = Control_kWh_daily + Injector_kWh_daily + Dehumidifier_kWh_daily + Fans_kWh_daily
    
    daily_cost_climate = Climate_energy * energy_price
    return(daily_cost_climate)

In [ ]:
def Energy_Plumbing(System,Plumbing_kit,energy_price):
    
    if System[0] == 'ZipTower':
        if System[1] < Plumbing_kit[2]: #If System rack units is less than Plumbing system capacity
            nPlumbing_kits = 1 # number of plumbing kits
        
        else:
            System[1] >= Plumbing_kit[2] # If system rack units is more than or equal to plumbing capacity
            nPlumbing_kits = math.ceil(System[1]/Plumbing_kit[2]) # Round number System units / plumbing capacity
        
        Pumps_farm_usage = nPlumbing_kits*Plumbing_kit[1]/1000 #kW of Pumps
        Pumps_kWh_daily = Pumps_farm_usage*24
        Pumps_daily_cost = Pumps_kWh_daily*energy_price
        
    else:
        System[0] != 'unknown'
        Pumps_daily_cost = 'unknown'
    
    return(Pumps_daily_cost)

In [11]:
def Energy_HVAC(energy_price,HVAC):
    
    Heating = HVAC[1,1] #power requirements
    Heating_time = HVAC[1,2}
    Heating_consumption = Heating*Heating_time/1000 #kWh per day for heating
    Ventilation = HVAC[2,1]
    Ventilation_time = HVAC[2,2] #ventilation requirements
    Ventilation_consumption = Ventilation*Ventilation_time/1000 #kWh per day for ventilation
    
    Energy_consumption = Heating_consumption + Ventilation_consumption #Energy consumption kWh per day
    costs = Energy_consumption*energy_price
    
    return(costs)

SyntaxError: invalid syntax (<ipython-input-11-957d6ff8591f>, line 1)

In [76]:
def Energy_Seedling(Seedling_kit,energy_price): #Should be split up into Pump. Lights and hours.
    
    
    Pumps_kWh_daily = Seedling_kit[1,1]*Seedling_kit[1,2]*Seedling_kit[1,3]/1000 # Pumps: Quantity x Watts x Hours /1000
    Lights_kWh_daily = Seedling_kit[2,1]*Seedling_kit[2,2]*Seedling_kit[2,3]/1000 # Lights: Quantity x Watts x Hours /1000
    
    Seedling_total_kWh = Pumps_kWh_daily + Lights_kWh_daily
    
    daily_cost_seedling = Seedling_total_kWh * energy_price
    
    return daily_cost_seedling

In [77]:
def Energy(Energy_Lights, Energy_Plumbing, Energy_Climate, Energy_HVAC, Energy_Seedling, Energy_pricing, size): # Rack or Tower Quantities
    if Energy_Lights != "unknown": 
        consumption = Energy_Lights + Energy_Plumbing + Energy_Climate + Energy_Plumbing + Energy_HVAC + Energy_Seedling #Daily consumption of energy
    else:
        Energy_Lights == "unknown"
        energy_demand = size*15 # 14-17 kWh per square-metre average energy for hydroponic plant factory (Xydis et al, 2017)
        consumption = energy_demand * 20 # ASSUMPTION: demand x 20 hours (as lights are run 18h, climate control and irrigation 24h)
    
    costs = consumption*Energy_pricing # daily
    return(costs)

In [79]:
def Water(System,RTQ,water_price,size): # Rack or Tower Quantities
    
    if System == "ZipTower":
        water_consumption = RTQ * 0.95 * 30 #Litres of water per tower per day (0.25 gallons) multiplied by month
        water_buffer = 1900 #Litres of water for buffer per month (500 gallons)
        water_consumption += water_buffer #Water consumption could be used here.
    else:
        water_consumptiona = size * 200 #Average from Agrilyst survey - 4 Gallons per sq ft per year
        water_consumption = water_consumptiona/12 # consumption per month
        
    water_cost = water_consumption * water_price #monthly cost of water
    return(water_cost)

In [15]:
def utilitiesM(Energy, Water, Internet, Carbon): #£ per month
    cost_of_utilities_monthly = (Energy*30) + Water + Internet + Carbon
    return cost_of_utilities_monthly
# Includes Energy consumption, water consumption, internet

In [16]:
def annualcosts(energystandingcharge,waterstandingcharge,tax)
    costs = energystandingcharge + waterstandingcharge + tax
    return(costs)

SyntaxError: invalid syntax (<ipython-input-16-987e13b691a3>, line 1)

In [ ]:
def rent(Annual_rent,Size,Location,Location_type): 
    if Annual_rent == "unknown":
        print("Size: ", Size, "m^2")
        print("Location: ", Location)
        print("Location Type: ", Location_type)
        print('Please enter an annual rent in pounds (£) for the spec')
    else:
        Monthly_rent = Annual_rent/12 
        return(Monthly_rent)



In [19]:
# def ShipPack

In [20]:
OpEx: int = 0
#def OpEx(days,Labour,Shipping,Utilities,Rent,Inputs,Packaging,Misc)
days = 365
for i in range(days):
    if i % 30 == 0:
        OpEx += labour(iSize, iStaff) 
        OpEx += rent(iAnnual_rent, iSize, iLocation, iLocation_type)
        OpEx += utilitiesM(Energy,Water,Internet, Carbon)
        OpEx +=
        OpEx +=
    else: i % 365 == 0 
        OpEx += annualcosts(ienergystandingcharge,iwaterstandingcharge,itax)
print(OpEx) 


#Operations = Labour + Shipping +Utilities + Rent + Inputs + Packaging
#Inputs = Seeds + Nutrients + Grow Media

SyntaxError: invalid syntax (<ipython-input-20-3fc537738e39>, line 9)